### 0. Импорт необходимых библиотек

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 1. Получение данных и первичной информации о них

In [61]:
data = pd.read_csv('datasets/games.csv')

data.info()
display(data.head())
display(data['Rating'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

В датасете 16715 строк записей. Данные разных типов. Присутствуют пропуски данных, некоторые данные представлены NaN.

### 2. Предобработка данных

#### 2.1 Замена названий столбцов

In [62]:
#  Переименовываем названия столбцов привoдя к нижнему регистру.
# renames = {'Name': 'name', 'Platform': 'platform', ...}; data = data.rename(columns = renames)
data.columns = data.columns.str.lower()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


#### 2.2 Преобразование типов данных

Преобразуем колонку "год выпуска"  к целому числу, а колонку с оценкой пользователей к вещественному, так как в ней наблюдаются дробные значения.

In [ ]:
# Год в целое

display(data['year_of_release'].unique())    # Посмотреть какие года есть
data['year_of_release'] = data['year_of_release'].fillna(0)
data['year_of_release'] = data['year_of_release'].astype('int')

# Оценка пользователей в float
data['user_score'] = pd.to_numeric(data['user_score'], errors='coerce')    # В случае NaN -  пропускаем и оставляем.

display(data.info())

#### 2.3 Обработка пропусков

In [ ]:
# Все пропуски по столбцам:
display(data.isna().sum())

for pos in ('name', 'genre'):
    display(data[pos].unique())

По два пропуска в столбцах 'name' и 'genre', большАя часть в столбцах с оценками и рейтингами.

In [ ]:
# Детально посмотрим что за пропуски в названиях и жанрах

for pos in ('name', 'genre'):
    display(data[pos].unique())

    missing_values = data[pos].isnull().sum()
    print(f"Количество пропусков в {pos}: {missing_values}")
    nan_indices = data[data[pos].isnull()].index
    print(f"Индексы пропусков в {pos}: {list(nan_indices)}")
display(data.loc[[659, 14244]])

array(['Wii Sports', 'Super Mario Bros.', 'Mario Kart Wii', ...,
       'Woody Woodpecker in Crazy Castle 5', 'LMA Manager 2007',
       'Haitaka no Psychedelica'], shape=(11559,), dtype=object)

Количество пропусков в name: 0
Индексы пропусков в name: []


array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy'], dtype=object)

Количество пропусков в genre: 0
Индексы пропусков в genre: []


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,Guitar Hero,PS2,2005,Misc,1.67,0.61,0.03,0.07,91.0,8.7,T
14244,Mashiro Iro Symphony: *mutsu-no-hana,PSP,2011,Adventure,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Две игры. Не оценивались и не имеют рейтинга. Продажи низкие. Предваарительное решение - удаляем.

In [ ]:
# Удаляем строки по индексам

data = data.drop(nan_indices).reset_index(drop=True)

Посмотрим в каких случаях проявляются значения NaN.

In [ ]:
print(data.query('critic_score.isna()').head(10))
# print(data.query('critic_score.isna() and user_score.isna() and rating.isna()').count())

                             name platform  year_of_release         genre  \
1               Super Mario Bros.      NES             1985      Platform   
4        Pokemon Red/Pokemon Blue       GB             1996  Role-Playing   
5                          Tetris       GB             1989        Puzzle   
9                       Duck Hunt      NES             1984       Shooter   
10                     Nintendogs       DS             2005    Simulation   
12    Pokemon Gold/Pokemon Silver       GB             1999  Role-Playing   
18              Super Mario World     SNES             1990      Platform   
20  Pokemon Diamond/Pokemon Pearl       DS             2006  Role-Playing   
21               Super Mario Land       GB             1989      Platform   
22            Super Mario Bros. 3      NES             1988      Platform   

    na_sales  eu_sales  jp_sales  other_sales  critic_score  user_score rating  
1      29.08      3.58      6.81         0.77           NaN         NaN

NaN появляется во все года, во всех платформах и жанрах, по самым разным продажам. Закономерности не видно, поэтому заменить чем-то конкретным трудно. Оставляем как есть. 

<b>'tbd'</b> -  to be determined - значения должны быть определены. Но пока это строковое значение, по каким-то причинам. Возможно пользователям не предлагалось оценить игру. Так как оценка <b>пока</b> не определена, заменим медианной оценкой по данному жанру игры.

In [125]:
# Заменяем 'tbd' на NaN
data['user_score'] = data['user_score'].replace('tbd', np.nan)

# Преобразуем в числовой формат
data['user_score'] = pd.to_numeric(data['user_score'], errors='coerce')
print(data['user_score'])
display(data['user_score'].isna().sum())


0        8.0
1        NaN
2        8.3
3        8.0
4        NaN
        ... 
16710    NaN
16711    NaN
16712    NaN
16713    NaN
16714    NaN
Name: user_score, Length: 16715, dtype: float64


np.int64(9125)

In [ ]:
# Заменяем в нужном столбце значения NaN на медианы для этого жанра
#  
for genre in data['genre'].unique():
    median = data[data['genre'] == genre]['user_score'].median()
    data.loc[(data['user_score'].isna()) & (data['genre'] == genre), 'user_score'] = median

data['user_score'].describe()

count    16713.000000
mean         7.310088
std          1.035561
min          0.000000
25%          7.100000
50%          7.400000
75%          7.800000
max          9.700000
Name: user_score, dtype: float64

#### 2.4 Суммарные продажи по регионам

In [ ]:
# ...
regions = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']
total_sales = {}

# Суммируем продажи для каждого региона
for region in regions:
    total_sales[region] = data[region].sum()

# Создаем DataFrame из суммы
total_sales_df = pd.DataFrame([total_sales])

display(total_sales_df)

,na_sales,eu_sales,jp_sales,other_sales
0,70948.62,69801.87,68542.94,68697.04
